In [2]:
%%time
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb
from sklearn.metrics import matthews_corrcoef

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.grid_search import RandomizedSearchCV
from catboost import CatBoostClassifier,CatBoostRegressor

from rgf.sklearn import RGFClassifier,RGFRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesClassifier,ExtraTreesRegressor
from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


C:\Users\royal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\royal\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Wall time: 35.5 s


In [2]:
%%time
os.chdir('C:\\Users\\royal\\Downloads\\Compressed\\Electricity_Production')
train=pd.read_csv('train_data.csv')
test=pd.read_csv('test_data.csv')

Wall time: 177 ms


In [3]:
train.head()

,T,V,P,RH,E
0,273.39,420.12,91.84,57.41,1778.12
1,195.26,248.88,92.29,35.21,1824.16
2,377.52,360.42,92.31,27.69,1761.72
3,171.60,244.98,91.61,56.99,1889.64
4,222.56,353.70,92.43,54.25,1806.68


In [60]:
##generate two extra features.
train['pv/t']=(train['P']*train['V'])/train['T']
train['pv']=train['P']*train['V']

test['pv/t']=(test['P']*test['V'])/test['T']
test['pv']=test['P']*test['V']

In [5]:
train.isnull().sum().sum()

0

In [4]:
##unique values in the columns.
for i in test.columns:
    print(i,len(test[i].unique()))

T 194
V 164
P 128
RH 191


In [5]:
Y1=train['E']
train1=train.drop(['E'],axis=1)
train1=train1.values
Y=Y1.values

test1=test.values

In [7]:
#scale training and testing data.
scaler = StandardScaler()
scaler.fit(train1)
train2 = scaler.transform(train1)
test2 = scaler.transform(test1)

In [53]:
#make 2 way interaction between all the columns
def Make_2way(X, Xt):
    columns_length=X.shape[1]
    for j in range (0,columns_length):
        for d in range (j+1,columns_length):  
            print(("Adding columns' interraction %d and %d" % (j, d) ))
            new_column_train=X[:,j]+X[:,d]
            new_column_test=Xt[:,j]+Xt[:,d]    
            X=np.column_stack((X,new_column_train))
            Xt=np.column_stack((Xt,new_column_test))
    return X, Xt

In [13]:
##make 2 and 3 way interaction between all the columns.
def Make_3way(X, Xt):
    columns_length=X.shape[1]
    for j in range (0,columns_length):
        for d in range (j+1,columns_length):  
            print(("Adding columns' interraction %d and %d" % (j, d) ))
            new_column_train=X[:,j]*X[:,d]
            new_column_test=Xt[:,j]*Xt[:,d]    
            X1=np.column_stack((X,new_column_train))
            Xt1=np.column_stack((Xt,new_column_test))
    
    for j in range (columns_length):
        for d in range (j+1,columns_length):  
            for m in range (d+1,columns_length):              
                print("Adding columns' interaction %d and %d and %d" % (j, d, m) )
                new_column_train=X[:,j]*X[:,d]*X[:,m]
                new_column_test=Xt[:,j]*Xt[:,d]*Xt[:,m]      
                X2=np.column_stack((X,new_column_train))
                Xt2=np.column_stack((Xt,new_column_test))
    X=np.column_stack((X1,X2))
    Xt=np.column_stack((Xt1,Xt2))
    return X, Xt

In [56]:
# st_train,st_test= Make_2way(train2, test2)
train2,test2= Make_2way(train1, test1)

Adding columns' interraction 0 and 1
Adding columns' interraction 0 and 2
Adding columns' interraction 0 and 3
Adding columns' interraction 1 and 2
Adding columns' interraction 1 and 3
Adding columns' interraction 2 and 3


In [57]:
scaler = StandardScaler()
scaler.fit(train2)
train2 = scaler.transform(train2)
test2 = scaler.transform(test2)

In [14]:
st_train,st_test= Make_3way(train1, test1)

Adding columns' interraction 0 and 1
Adding columns' interraction 0 and 2
Adding columns' interraction 0 and 3
Adding columns' interraction 1 and 2
Adding columns' interraction 1 and 3
Adding columns' interraction 2 and 3
Adding columns' interaction 0 and 1 and 2
Adding columns' interaction 0 and 1 and 3
Adding columns' interaction 0 and 2 and 3
Adding columns' interaction 1 and 2 and 3


In [76]:
##generate the cross validation fold.
from sklearn.cross_validation import StratifiedKFold
SEED=42
# clf = lgb.LGBMRegressor() 
# clf = lgb.LGBMRegressor(max_depth=6,learning_rate=0.0716,n_estimators=128,num_leaves=24,reg_alpha=1.7250,reg_lambda=0.0888,subsample=0.6361,colsample_bytree=0.9365)
st_train = train2
st_test = test2
# clf=CatBoostRegressor(iterations=80)
clf = xgb.XGBRegressor(gamma = 0.76,learning_rate = 0.039,max_depth = 7,min_child_weight = 9.6754,n_estimators = 1100,subsample = 0.8,colsample_bytree = 0.99)
# clf = xgb.XGBRegressor(gamma = 0.7412,learning_rate = 0.0290,max_depth = 8,min_child_weight = 9.6754,n_estimators = 999) #96.00
# clf=ExtraTreesRegressor(n_estimators=10000, max_depth=9,  min_samples_leaf=1,  n_jobs=30, random_state=1)
# clf = RGFRegressor(max_leaf=500,algorithm="RGF",test_interval=100, loss="LS")

fold = 5
cv = KFold(n_splits=fold, shuffle=True, random_state=239)
X_preds = np.zeros(st_train.shape[0])
preds = np.zeros(st_test.shape[0])
for i, (tr, ts) in enumerate(cv.split(st_train)):
    print(ts.shape)
    mod = clf.fit(st_train[tr], Y[tr])
    X_preds[ts] = mod.predict(st_train[ts])
    preds += mod.predict(st_test)
    print("fold {}, ROC AUC: {:.3f}".format(i, r2_score(Y[ts], X_preds[ts])))
score = r2_score(Y, X_preds)
print(score)
preds1 = preds/fold

(400,)
fold 0, ROC AUC: 0.946
(400,)
fold 1, ROC AUC: 0.956
(400,)
fold 2, ROC AUC: 0.952
(400,)
fold 3, ROC AUC: 0.952
(400,)
fold 4, ROC AUC: 0.944
0.9499539632007946


In [73]:
np.savetxt('submission.csv',preds1)

In [74]:
##save cross validation fold result
filename = 'xgb1_'+'{:.5f}'.format(score)
np.savetxt('oof+test//'+filename+'_train.csv',X_preds)
np.savetxt('oof+test//'+filename+'_test.csv',preds1)

In [70]:
#xgboost bayesian optimization
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization

def xgboostcv(max_depth,learning_rate,n_estimators,gamma,min_child_weight,subsample,colsample_bytree,max_delta_step):
    return cross_val_score(xgb.XGBRegressor(max_depth=int(max_depth),learning_rate=learning_rate,n_estimators=int(n_estimators),
                                             silent=True,nthread=-1,gamma=gamma,min_child_weight=min_child_weight,
                                           subsample=subsample,colsample_bytree=colsample_bytree),
                           train2,Y,"r2",cv=5).mean()

xgboostBO = BayesianOptimization(xgboostcv,{'max_depth': (5, 10),'learning_rate': (0.01, 0.3),'n_estimators': (50, 1200),
                                  'gamma': (0.01,1.0),'min_child_weight': (2, 10),
                                            'subsample': (0.6, 0.8),'colsample_bytree' :(0.5, 0.99)})


xgboostBO.maximize()
print('-'*53)
print('Final Results')
print('XGBOOST: %f' % xgboostBO.res['max']['max_val'])

Initialization
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   subsample | 
    1 | 00m05s | -744.75915 |             0.7669 |    0.7583 |          0.0316 |           0.0635 |      7.5813 |             9.7865 |      1037.8835 |      0.6129 | 
    2 | 00m07s | -746.41719 |             0.8973 |    0.7430 |          0.0325 |           0.0018 |      7.2371 |             9.7400 |      1039.7247 |      0.7268 | 
    3 | 00m04s | -744.21274 |             0.6630 |    0.7554 |          0.0285 |           0.0939 |      7.8603 |             9.8746 |      1024.2118 |      0.6412 | 
    4 | 00m05s | -744.69354 |             0.8703 |    0.7597 |          0.0282 |           0.0763 |      7.8091 |             9.7868 |      1004.1146 |

C:\Users\royal\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.30964291]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


    9 | 00m51s | -744.90758 |             0.9900 |    0.7600 |          0.0190 |           0.1000 |      7.0000 |             9.9000 |       999.0000 |      0.6000 | 
   10 | 01m01s | -744.88262 |             0.9900 |    0.7600 |          0.0190 |           0.1000 |      9.0000 |             9.9000 |      1015.2039 |      0.6000 | 
   11 | 00m52s | -744.84830 |             0.5000 |    0.7600 |          0.0190 |           0.1000 |      9.0000 |             9.9000 |      1037.8233 |      0.6000 | 
   12 | 00m51s | -743.16258 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      9.0000 |             9.9000 |      1032.3008 |      0.6000 | 
   13 | 00m36s | -743.05065 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      9.0000 |             9.9000 |      1067.5259 |      0.6000 | 
   14 | 00m48s | -743.09223 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      9.0000 |             9.9000 |      1054.0484 |      0.6000 |

C:\Users\royal\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([3.89144981]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\Users\royal\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.42495084]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   24 | 01m05s | -742.94513 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      7.0000 |             9.6754 |      1100.0000 |      0.8000 | 


...

   26 | 01m10s | -743.24254 |             0.5000 |    0.7600 |          0.0390 |           0.1000 |      7.0000 |             9.6754 |      1007.5049 |      0.6000 | 
   27 | 01m07s | -743.16258 |             0.5000 |    0.7600 |          0.0390 |           0.1000 |      7.6721 |             9.8191 |      1032.7841 |      0.6000 | 
   28 | 01m08s | -743.12101 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      9.0000 |             9.9000 |      1045.9328 |      0.8000 | 


C:\Users\royal\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.27031007]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   29 | 00m59s | -743.06025 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      9.0000 |             9.6754 |      1064.2740 |      0.6000 | 
   30 | 01m07s | -743.07944 |             0.9900 |    0.7600 |          0.0390 |           0.1000 |      9.0000 |             9.6754 |      1058.7920 |      0.6000 | 


C:\Users\royal\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([1.01724488]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


-----------------------------------------------------
Final Results
XGBOOST: -742.945129


In [36]:
#lightgbm bayesian optimization
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization

def xgboostcv(max_depth,learning_rate,n_estimators,num_leaves,reg_alpha,reg_lambda,subsample,colsample_bytree):
    return cross_val_score(lgb.LGBMRegressor(max_depth=int(max_depth),learning_rate=learning_rate,n_estimators=int(n_estimators),
                                             silent=True,nthread=-1,num_leaves=int(num_leaves),reg_alpha=reg_alpha,
                                           reg_lambda=reg_lambda,subsample=subsample,colsample_bytree=colsample_bytree),
                           train2,Y,"r2",cv=5).mean()

xgboostBO = BayesianOptimization(xgboostcv,{'max_depth': (5, 10),'learning_rate': (0.001, 0.1),'n_estimators': (10, 300),
                                  'num_leaves': (6,30),'reg_alpha': (1, 3),'reg_lambda': (0, 0.1),
                                            'subsample': (0.6, 0.8),'colsample_bytree' :(0.5, 0.99)})

xgboostBO.maximize()
print('-'*53)
print('Final Results')
print('XGBOOST: %f' % xgboostBO.res['max']['max_val'])

Initialization
-------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   learning_rate |   max_depth |   n_estimators |   num_leaves |   reg_alpha |   reg_lambda |   subsample | 
    1 | 00m00s |    0.93953 |             0.9255 |          0.0782 |      6.9504 |        49.4098 |       8.4761 |      1.9180 |       0.0248 |      0.7018 | 
    2 | 00m01s |    0.94578 |             0.9365 |          0.0716 |      6.0354 |       128.7693 |      24.6596 |      1.7250 |       0.0888 |      0.6361 | 
    3 | 00m00s |    0.80776 |             0.6051 |          0.0611 |      8.0006 |        28.4322 |      15.5507 |      1.6885 |       0.0643 |      0.6271 | 
    4 | 00m00s |    0.94214 |             0.9681 |          0.0988 |      6.0960 |        37.9167 |      24.7322 |      2.2527 |       0.0432 |      0.6931 | 
    5 | 00m02s |    0.92866 |   

In [71]:
##get the max accuracy hyperparameter.
xgboostBO.res['max']

{'max_params': {'colsample_bytree': 0.99,
  'gamma': 0.76,
  'learning_rate': 0.039,
  'max_delta_step': 0.1,
  'max_depth': 7.0,
  'min_child_weight': 9.6754,
  'n_estimators': 1100.0,
  'subsample': 0.8},
 'max_val': -742.9451286952915}